<!--BOOK_INFORMATION-->
*This notebook contains an excerpt from the <a href="http://shop.oreilly.com/product/0636920034919.do">Python Data Science Handbook</a> by Jake VanderPlas; the content is available on <a href="https://github.com/jakevdp/PythonDataScienceHandbook">GitHub</a>.*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

Adapded for class presentation by Claudio Sartori - University of Bologna

# Introducing Scikit-Learn

<a href="http://scikit-learn.org">Scikit-Learn</a>
- package that provides efficient versions of a large number of common algorithms
- clean, uniform, and streamlined API
- very useful and complete online documentation.
    - once you understand the basic use and syntax of Scikit-Learn for one type of model, switching to a new model or algorithm is very straightforward
    
## Contents

- *Estimator* API
- *Examples*
- Upload and explore the Iris dataset https://archive.ics.uci.edu/dataset/53/iris 
- prepare _features_ and _target_
- split the supervised data into train and test
- fit a classifier on the train part
- predict the train labels and evaluate the accuracy
- prdict the test labels and evaluate the accuracy

In [2]:
import seaborn as sns
import pandas as pd
train_size = 0.67
random_state = 42 # any positive integer is acceptable, it is necessary for **reproducibility

In [5]:
df = pd.read_csv("../Lab2/iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
df.head()

sepal_length  sepal_width  petal_length  petal_width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa

#### Features matrix

The part of the data matrix containing the ``unsupervised attributes``

Usually in *scikit-learn* documentation referred as ``X``

Can be a:
- two-dimensional numpy array with shape ``[n_samples, n_features]``
- SciPy ``sparse matrix``
- Pandas ``DataFrame``

The matrix cases require uniform data types in columns


#### Target array

*label* or *target* array, by convention usually called ``y``
- usually one dimensional, with length ``n_samples``, 
- generally contained in a NumPy array or Pandas ``Series``.
- may have continuous numerical values, or discrete classes/labels
- usually it the quantity we want to *predict from the data*
    - in statistical terms, it is the dependent variable

In the example we may wish to construct a model that can predict the species of flower based on the other measurements

The measurements of the flower components are the ``features array``

The ``class`` column can be considered the target array

Assign the target column name to a variable `target

### Visualization

Use Seaborn (see [Visualization With Seaborn](04.14-Visualization-With-Seaborn.ipynb)) to visualize the data

Below we need to prepare the environment for plotting information on the dataset.

1. import `seaborn` giving it the 'nickname' `sns`
1. call the `pairplot` function of `seaborn` on the `iris` dataset, with parameters 
    - `hue = target`, this sets the meaning of the color in the plot of the points of the dataset
    - `height = 2`, this sets the size of the plots
    
--> insert your code in a new cell below this one

In [16]:
import seaborn as sns

sns.pairplot(df, hue='class', height=2)

For use in Scikit-Learn, we will extract the features matrix and target array from the ``DataFrame``. We can do this using some of the Pandas ``DataFrame`` operations discussed in the [Chapter 3](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html) of the above mentioned book.

For example, the `.drop` method allows to drop a column or row by name; remember to specify the axis to use, which is 1 for columns.

### Preparing features and target
Store in X the content of `iris` excluding the column `class`. Verify the shape

--> insert your code in a new cell below this one

In [26]:
X = df.drop(["class"], axis=1)

Store in `y` the `target` of `iris`. Verify the shape

--> insert your code in a new cell below this one

In [27]:
y = df["class"]

To summarize, the expected layout of features and target values is visualized in the following diagram:
![Figure](figures/05.02-samples-features.png "Features and target")
With this data properly formatted, we can move on to consider the estimator API of Scikit-Learn:

## Scikit-Learn's Estimator API

The Scikit-Learn API is designed with the following guiding principles in mind, as outlined in the [Scikit-Learn API paper](http://arxiv.org/abs/1309.0238):

- *Consistency*: All objects share a common interface drawn from a limited set of methods, with consistent documentation.

- *Inspection*: All specified parameter values are exposed as public attributes.

- *Limited object hierarchy*: Only algorithms are represented by Python classes; datasets are represented
  in standard formats (NumPy arrays, Pandas ``DataFrame``s, SciPy sparse matrices) and parameter
  names use standard Python strings.

- *Composition*: Many machine learning tasks can be expressed as sequences of more fundamental algorithms,
  and Scikit-Learn makes use of this wherever possible.

- *Sensible defaults*: When models require user-specified parameters, the library defines an appropriate default value.

In practice, these principles make Scikit-Learn very easy to use, once the basic principles are understood.
Every machine learning algorithm in Scikit-Learn is implemented via the Estimator API, which provides a consistent interface for a wide range of machine learning applications.

## Hyperparameters
The machine learning algorithms are designed to learn from the data the _parameters_ that will be used at run time by the algorithms implementing the tasks to perform at the best on data similar to those used in learning. 

For example, a _decision tree_ (and in particular all the tests placed in the nodes) are the parameters of a _decision tree classifier_

The learning process is also controlled by other parameters (e.g. to control the _overfitting_ ) which cannot be direcly learned from the data, but are chosen _before_ the learning process. Those are called __hyperparameters__

### Basics of the API

Most commonly, the steps in using the Scikit-Learn estimator API are as follows
(we will step through a handful of detailed examples in the sections that follow).

1. Choose a class of model by importing the appropriate estimator class from Scikit-Learn.
2. Choose model hyperparameters by instantiating this class with desired values.
    - or in the first attempt use the default values
3. Arrange data into a features matrix and target vector following the discussion above.
4. Fit the model to your data by calling the ``fit()`` method of the model instance.
5. Apply the Model to new data:
   - For supervised learning, often we predict labels for unknown data using the ``predict()`` method.
   - For unsupervised learning, we often transform or infer properties of the data using the ``transform()`` or ``predict()`` method.


### Supervised learning example: Iris classification

Let's take a look at another example of this process, using the Iris dataset we discussed earlier.
Our question will be this: given a model trained on a portion of the Iris data, how well can we predict the remaining labels?

For this task, we will use the _Decision Tree_ algorithm, with the standard parameter values.
We would like to evaluate the model on data it has not seen before, and so we will split the data into a *training set* and a *testing set*.
This could be done by hand, but it is more convenient to use the ``train_test_split`` utility function

1. Import the method `train_test_split` from `sklearn.model_selection`<br>
2. Generate the variables `Xtrain, Xtest, ytrain, ytest` by calling the function `train_test_split` with parameters `X` and `y`, and the additional parameter `random_state`, this must be set to a positive integer, for reproducibility<br>
3. Show the shape of the resulting variables<br>


--> insert your code in a new cell below this one

In [72]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=random_state)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(112, 4) (38, 4) (112,) (38,)


With the data arranged, we can follow our recipe to predict the labels:
1. choose the model class, it will be `DecisionTreeClassifier`, imported from `sklearn.tree`<br>
2. instantiate the `model` as a `DecisionTreeClassifier` whithout any hyperparameter, we will use the defaults<br>
3. fit the `model` to data, calling its method `fit` with parameters `Xtrain, ytrain`<br>
4. predict the target `ytrain_dt` using the `predict` method of `model` on the `Xtrain` data

--> insert your code in a new cell below this one

In [73]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier()
model.fit(Xtrain, ytrain)


DecisionTreeClassifier()

We can use the ``accuracy_score`` utility to see the fraction of predicted training set labels that match their true value.

Import the `accuracy_score` from `sklearn.metrics` and call it on `ytrain, ytrain_dt`

--> insert your code in a new cell below this one

In [74]:
ytrain_dt = model.predict(Xtrain)

train_acc = accuracy_score(ytrain, ytrain_dt)
print(f"train accuracy score: {train_acc*100}%")

train accuracy score: 100.0%


In [75]:
ytest_dt = model.predict(Xtest)

test_acc = accuracy_score(ytest, ytest_dt)
print(f"test accuracy score: {test_acc*100}%")

test accuracy score: 100.0%


## Show the Decision Tree
To show the Decision Tree we will need a few imports

`from matplotlib import pyplot`<br>
`from sklearn.tree import plot_tree`<br>
`from matplotlib.pyplot import figure`

We will start setting the _figure size_ with the `figure` function, taking as argument `figsize` and a list of two values in inches, try and error for the measures you like.

We will then use the `plot_tree` function of `sklearn.tree`. It takes as argument the *fitted model*\, in our case `model` and several arguments to control how the tree is displayed.

I suggest the arguments below, you can try freely configurations and omissions of the parameters, to use the defaults. The parameters must follow the model variable and be separated by commas, the order is not relevant, since the parameters are named.

`filled=True`<br>
`feature_names = ['sepal length', 'sepal width', 'petal length', 'petal width']`<br>
`class_names = ['setosa', 'versicolor', 'virginica']`<br>
`rounded = True`<br>
`proportion = True`<br>

--> insert your code in a new cell below this one

In [76]:
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.tree import plot_tree

figure(figsize=(10, 10))
plot_tree(model, 
            filled=True,
            feature_names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"],
            class_names=['setosa', 'versicolor', 'virginica'],
            rounded=True,
            proportion=True
        )

[Text(0.3076923076923077, 0.9285714285714286, 'petal_width <= 0.8\ngini = 0.666\nsamples = 100.0%\nvalue = [0.312, 0.348, 0.339]\nclass = versicolor'),
 Text(0.23076923076923078, 0.7857142857142857, 'gini = 0.0\nsamples = 31.2%\nvalue = [1.0, 0.0, 0.0]\nclass = setosa'),
 Text(0.2692307692307693, 0.8571428571428572, 'True  '),
 Text(0.38461538461538464, 0.7857142857142857, 'petal_length <= 4.75\ngini = 0.5\nsamples = 68.8%\nvalue = [0.0, 0.506, 0.494]\nclass = versicolor'),
 Text(0.34615384615384615, 0.8571428571428572, '  False'),
 Text(0.15384615384615385, 0.6428571428571429, 'petal_width <= 1.65\ngini = 0.056\nsamples = 31.2%\nvalue = [0.0, 0.971, 0.029]\nclass = versicolor'),
 Text(0.07692307692307693, 0.5, 'gini = 0.0\nsamples = 30.4%\nvalue = [0.0, 1.0, 0.0]\nclass = versicolor'),
 Text(0.23076923076923078, 0.5, 'gini = 0.0\nsamples = 0.9%\nvalue = [0.0, 0.0, 1.0]\nclass = virginica'),
 Text(0.6153846153846154, 0.6428571428571429, 'petal_width <= 1.75\ngini = 0.21\nsamples = 37.5

## Second experiment
Repeat the operations above on the Wine Quality dataset https://archive.ics.uci.edu/dataset/186/wine+quality